In [5]:
import numpy
import urllib
import scipy.optimize
import random
from sklearn import svm

In [6]:
def parseData(fname):
  for l in urllib.urlopen(fname):
    yield eval(l)


In [ ]:
print "Reading data..."
data = list(parseData("http://jmcauley.ucsd.edu/cse255/data/amazon/book_descriptions_50000.json"))
print "done"

In [4]:
data[2]

{'asin/ID': '157120315X',
 'categories': ["Children's Books", 'Crafts'],
 'description': 'quiltmaker magazine one of americas most prestigious quilting publications also created the books paper piecing picnic and paper piecing potpourri',
 'url': 'http://www.amazon.com/dp/157120315X'}

### Naive bayes

In [6]:
### Naive bayes to determine p(childrens book | mentions wizards and mentions witches) ###

# p(childrens book)
prior = ["Children's Books" in b['categories'] for b in data]
prior = sum(prior) * 1.0 / len(prior)
prior_neg = 1 - prior

In [7]:

# p(mentions wizards | is childrens)
p1 = ['wizard' in b['description'] for b in data if "Children's Books" in b['categories']]
p1 = sum(p1) * 1.0 / len(p1)

# p(mentions wizards | isn't childrens)
p1_neg = ['wizard' in b['description'] for b in data if not ("Children's Books" in b['categories'])]
p1_neg = sum(p1_neg) * 1.0 / len(p1_neg)

# p(mentions witches | is childrens)
p2 = ['witch' in b['description'] for b in data if "Children's Books" in b['categories']]
p2 = sum(p2) * 1.0 / len(p2)

# p(mentions witches | isn't childrens)
p2_neg = ['witch' in b['description'] for b in data if not ("Children's Books" in b['categories'])]
p2_neg = sum(p2_neg) * 1.0 / len(p2_neg)

In [9]:
# Prediction

score = prior * p1 * p2
score_neg = prior_neg * p1_neg * p2_neg

In [10]:
score / score_neg

0.3793836356818883

In [12]:
# Actual ('non-naive') probability

p = ["Children's Books" in b['categories'] for b in data if 'witch' in b['description'] and 'wizard' in b['description']]
p = sum(p) * 1.0 / len(p)
p

0.3157894736842105

### SVM

In [8]:
print "Reading data..."
data = list(parseData("http://jmcauley.ucsd.edu/cse255/data/amazon/book_images_5000.json"))
print "done"

Reading data...
done


In [1]:
X = [b['image_feature'] for b in data]
y = ["Children's Books" in b['categories'] for b in data]

NameError: name 'data' is not defined

In [12]:
X_train = X[:2500]
y_train = y[:2500]

X_test = X[2500:]
y_test = y[2500:]

In [13]:
# Create a support vector classifier object, with regularization parameter C = 1000
clf = svm.SVC(C=1000, kernel='linear')
clf.fit(X_train, y_train)

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [15]:
train_predictions = clf.predict(X_train)
test_predictions = clf.predict(X_test)

In [16]:
train_predictions[:10]

array([False, False,  True, False, False, False, False, False, False,
       False])

In [18]:
train_correct = train_predictions == y_train
test_correct = test_predictions == y_test

In [20]:
train_acc = sum(train_correct) / len(train_correct)
train_acc

1

In [21]:
y_test[:10]

[False, False, False, False, True, False, False, False, False, False]